**Reinforcement Learning: Individual Assignment**

Name: Basma Reda Shaban Abd-Elsalam Abd-Elwahab


Student number: 300327209

Email: babde014@uottawa.ca

#Importing important libraries


In [1]:
!pip install cmake 'gym[atari]' scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from numpy import array #to deal with arrays
import math #to deal with math functions
from math import inf
from numpy.linalg import norm
import gym #for developing reinforcement learning environments
import random 
import numpy as np
from IPython.display import clear_output #to produce clear output
from time import sleep #to sleep for a time 


**Define our reinforcement learning environment**

Taxi-V3 environment>>>>>Teach a Taxi Cab to drive around with Q-Learning

In [3]:
env = gym.make("Taxi-v3")
#env1 = gym.make("MountainCar-v0")

In [4]:
env.reset() #A new environment is generated
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [5]:
#print the 6 actions that the taxi can take, and the 500 states of the taxi
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


In [6]:
env.action_space.n, env.observation_space.n

(6, 500)

#Problem 1: Turning the code into a module of functions that can use multiple environments

In [7]:
#env.s = 328  # set environment to illustration's state
def set_time(env):
  epochs = 0   #set the number of epochs
  penalties = 0 #set the number of penalties
  reward = 0 #set the number of reward
  frames = [] # list of frames for animation
  done = False #set done to false

  while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    if reward == -10:
        penalties += 1
    # Put the rendered frame into dictionary for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    #print the time taken to reach and penalities of the agent    
  print("Timesteps taken: {}".format(epochs))
  print("Penalties incurred: {}".format(penalties))
  return frames


In [8]:
frames = set_time(env)

Timesteps taken: 200
Penalties incurred: 68


In [9]:
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        #print(frame['frame'].getvalue())
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

Timestep: 200
State: 141
Action: 3
Reward: -1


**Start training the agent of Taxi Driver**

In [10]:
def TrainAgent(env):
  #define q_table
  q_table = np.zeros([env.observation_space.n, env.action_space.n])
  #define the hyperparameters
  epsilon = 0.5
  alpha = 0.1
  gamma = 0.6
  #plotting the matrix
  all_epochs = []
  all_penalties = []
  table = []
  for i in range(0, 15100):
    state = env.reset()
    epochs = 0
    penalties = 0
    reward = 0
    #set the done state to false
    done = False
    #while loop to check the state of the agent
    while done == False:
    #select the action:
    # take a random number
        if random.uniform(0, 1) < epsilon: 
          action = env.action_space.sample() # Explore action space
        else:
          action = np.argmax(q_table[state,:]) # Exploit learned values
            # Then we perform the action and receive the feedback from the environment
          new_state, reward, done, info = env.step(action)
            # Finally we learn from the experience by updating the Q-value of the selected action
          update = reward + (gamma*np.max(q_table[new_state,:])) - q_table[state, action]
          q_table[state,action] += alpha*update 
          if ( reward == -10):
            penalties += 1
          state = new_state
          epochs += 1
    if( i % 100 ==0):
      clear_output(wait=True)
      print(f"Episode: {i}")

  return q_table
q_table = TrainAgent(env)

Episode: 15000


In [11]:
print("Q table values are:\n", q_table )

Q table values are:
 [[ 0.          0.          0.          0.          0.          0.        ]
 [-2.28873104 -2.29275445 -2.28980572 -2.29275445 -2.27325184 -2.8816    ]
 [-1.72727388 -1.73474572 -1.73864537 -1.73474572 -0.7504     -1.96      ]
 ...
 [-1.14478405 -1.09266831 -1.14478405 -1.16369882 -1.96       -1.96      ]
 [-1.9249516  -1.93872851 -1.9249516  -1.92446755 -1.96       -1.96      ]
 [-0.196      -0.196      -0.196       0.24968    -1.         -1.        ]]


**Evaluate and launch the agent**

In [12]:
def launch_game(q_table, env):
  #define total epochs and penalities
  total_epochs = 0
  total_penalties = 0
  #set the number of episodes to 100
  episodes = 100
  for x in range(episodes):
    # define initial state
    state = env.reset()
    epochs = 0
    penalties = 0 
    reward = 0
   #set done state ro false
    done = False
    while done == False:
        # Take the action (index) with the maximum expected discounted future reward given that state
        action = np.argmax(q_table[state,:])
        state, reward, done, info = env.step(action)
        if reward == -10:
              penalties += 1

        epochs += 1
        #increment total penalities of the agent and total epochs 
        total_penalties += penalties
        total_epochs += epochs
  print(f"Average timesteps per episode: {total_epochs / episodes}")
  print(f"Average penalties per episode: {total_penalties / episodes}")
launch_game(q_table, env)

Average timesteps per episode: 300.33
Average penalties per episode: 0.0


#Problem 2: Tuning alpha, gamma, and/or epsilon using a decay over episodes

In [13]:
def HyperParametersTuning(env,epsilon, alpha, gamma):
  q_table = np.zeros([env.observation_space.n, env.action_space.n])
  #define the hyperparameters
  epsilon = epsilon
  alpha = alpha
  gamma = gamma
  
 #plotting the matrix  
  all_epochs = []
  all_penalties = []
  table = []
  for i in range(1, 15100):
    state = env.reset()
    epochs = 0
    penalties = 0
    reward = 0      
       #set the done state to false
    done = False
    #while loop to check the state of the agent
    while not done:
      if random.uniform(0, 1) < epsilon:
         action = env.action_space.sample() 
      else:
          action = np.argmax(q_table[state]) 
          next_state, reward, done, info = env.step(action) 
          
          old_value = q_table[state, action]
          new_state = np.max(q_table[next_state])
          
          new_value = (1 - alpha) * old_value + alpha * (reward + gamma * new_state)
          q_table[state, action] = new_value
          table.append(q_table[state, action])
          epsilon = epsilon - (0.0001*epsilon)
          alpha = alpha - (0.0001*alpha)
          gamma = gamma - (0.0001*gamma)
          if epsilon<= 0:
            epsilon = 0.1
          if alpha<=0:
            alpha = 0.1
          if gamma<=0:
            gamma = 0.6

          if reward == -10:
              penalties += 1

          state = next_state
          epochs += 1
          
      if i % 100 == 0:
          clear_output(wait=True)
          print(f"Episode: {i}")
  return q_table

In [14]:
new_env1 = gym.make("Taxi-v3")
new_q_table_1 = HyperParametersTuning(new_env1,0.1,0.1,0.1)
launch_game(new_q_table_1,new_env1)

Episode: 15000
Average timesteps per episode: 20100.0
Average penalties per episode: 0.0


#Problem 3: Implementing a grid search to discover the best hyperparameters

In [15]:
def TrainingGridSearch(env,epsilon, alpha, gamma):
  #define q_table
  q_table = np.zeros([env.observation_space.n, env.action_space.n])
  #define the hyperparameters
  epsilon = 0.1
  alpha = 0.1
  gamma = 0.6
  #plotting the matrix
  all_epochs = []
  all_penalties = []
  table = []
  for i in range(0, 15100):
    state = env.reset()
    epochs = 0
    penalties = 0
    reward = 0
    #set the done state to false
    done = False
    #while loop to check the state of the agent
    while done == False:
    #select the action:
    # take a random number
        if random.uniform(0, 1) < epsilon: 
          action = env.action_space.sample() # Explore action space
        else:
          action = np.argmax(q_table[state,:]) # Exploit learned values
            # Then we perform the action and receive the feedback from the environment
          new_state, reward, done, info = env.step(action)
            # Finally we learn from the experience by updating the Q-value of the selected action
          update = reward + (gamma*np.max(q_table[new_state,:])) - q_table[state, action]
          q_table[state,action] += alpha*update 
          if ( reward == -10):
            penalties += 1
          state = new_state
          epochs += 1
    if( i % 100 ==0):
      clear_output(wait=True)
      print(f"Episode: {i}")

  return q_table,epsilon, alpha, gamma

In [16]:
def EvaluateGridSearch(q_table, env):
  #define total epochs and penalities
  total_epochs = 0
  total_penalties = 0
  #set the number of episodes to 100
  episodes = 100
  for x in range(episodes):
    # define initial state
    state = env.reset()
    epochs = 0
    penalties = 0 
    reward = 0
   #set done state ro false
    done = False
    while done == False:
        # Take the action (index) with the maximum expected discounted future reward given that state
        action = np.argmax(q_table[state,:])
        state, reward, done, info = env.step(action)
        if reward == -10:
              penalties += 1

        epochs += 1
        #increment total penalities of the agent and total epochs 
        total_penalties += penalties
        total_epochs += epochs
        TAvg = total_epochs / episodes
        pAvg = total_penalties / episodes
  print(f"Average timesteps per episode: {TAvg}")
  print(f"Average penalties per episode: {pAvg}")
  return TAvg, pAvg

In [17]:
def grid_search(parm,env):
  time_steps = 15000
  penalties = 15000
  GridSearchHyperParameters = parm
  for i in GridSearchHyperParameters['alpha']:
    for j in GridSearchHyperParameters['gamma']:
      for k in GridSearchHyperParameters['epsilon']:
        q_table,alpha,gamma,epsilon = TrainingGridSearch(env,alpha=i,gamma=j,epsilon=k)
        TAvg,pAvg = EvaluateGridSearch(q_table,env)
        if TAvg<= time_steps:
          if pAvg <= penalties:
            time_steps = TAvg
            penalties = pAvg
            bestparameter = {'alpha':alpha,'gamma':gamma,'epsilon':epsilon,'Time':TAvg,'penalties':pAvg}
  return bestparameter

In [18]:
GridSearchHyperParameters = {'alpha':[0.1,0.2,0.3],'gamma':[0.1,0.2,0.3],'epsilon':[0.1,0.2,0.3]}
grid_search(GridSearchHyperParameters,env)

Episode: 15000
Average timesteps per episode: 696.65
Average penalties per episode: 0.0


{'Time': 289.95, 'alpha': 0.1, 'epsilon': 0.6, 'gamma': 0.1, 'penalties': 0.0}

In [19]:
GridSearchHyperParameters = {'alpha':[0.4,0.5,0.6],'gamma':[0.4,0.5,0.6],'epsilon':[0.4,0.5,0.6]}
grid_search(GridSearchHyperParameters,env)

Episode: 15000
Average timesteps per episode: 695.34
Average penalties per episode: 0.0


{'Time': 294.45, 'alpha': 0.1, 'epsilon': 0.6, 'gamma': 0.1, 'penalties': 0.0}

In [20]:
GridSearchHyperParameters = {'alpha':[0.7,0.8,0.9],'gamma':[0.7,0.8,0.9],'epsilon':[0.7,0.8,0.9]}
grid_search(GridSearchHyperParameters,env)

Episode: 15000
Average timesteps per episode: 292.98
Average penalties per episode: 0.0


{'Time': 93.86, 'alpha': 0.1, 'epsilon': 0.6, 'gamma': 0.1, 'penalties': 0.0}

#Apply the training and evaluation agent functions to a different Q learning environment called 'FrozenLake-v0' environment

In [21]:
env2 = gym.make("FrozenLake-v0")

In [22]:
q_table2 = TrainAgent(env2)

Episode: 15000


In [23]:
launch_game(q_table2, env2)

Average timesteps per episode: 258.87
Average penalties per episode: 0.0


In [24]:
new_env2 = gym.make("FrozenLake-v0")
new_q_table_2 = HyperParametersTuning(new_env2,0.1,0.1,0.1)
launch_game(new_q_table_2,new_env2)

Episode: 15000
Average timesteps per episode: 248.34
Average penalties per episode: 0.0


In [25]:
GridSearchHyperParameters = {'alpha':[0.1,0.2,0.3],'gamma':[0.1,0.2,0.3],'epsilon':[0.1,0.2,0.3]}
grid_search(GridSearchHyperParameters,env2)

Episode: 15000
Average timesteps per episode: 316.91
Average penalties per episode: 0.0


{'Time': 149.1, 'alpha': 0.1, 'epsilon': 0.6, 'gamma': 0.1, 'penalties': 0.0}

In [26]:
GridSearchHyperParameters = {'alpha':[0.4,0.5,0.6],'gamma':[0.4,0.5,0.6],'epsilon':[0.4,0.5,0.6]}
grid_search(GridSearchHyperParameters,env2)

Episode: 15000
Average timesteps per episode: 254.93
Average penalties per episode: 0.0


{'Time': 197.71, 'alpha': 0.1, 'epsilon': 0.6, 'gamma': 0.1, 'penalties': 0.0}

In [27]:
GridSearchHyperParameters = {'alpha':[0.7,0.8,0.9],'gamma':[0.7,0.8,0.9],'epsilon':[0.7,0.8,0.9]}
grid_search(GridSearchHyperParameters,env2)

Episode: 15000
Average timesteps per episode: 281.44
Average penalties per episode: 0.0


{'Time': 192.58, 'alpha': 0.1, 'epsilon': 0.6, 'gamma': 0.1, 'penalties': 0.0}